In [9]:
from skbio import TreeNode
import pandas as pd
from Bio import SeqIO
import networkx as nx
import pickle
import copy
import os


## First, we are going to open our metadata file (which says which samples as associated with which ages), filter the rows of the table we want, and prepare a dataframe for our dataset.

In [3]:
import os
print(os.listdir())

['BIG_GRAPH_ages_edge_weights_and_node_int_ids.pickle', 'feature-table.biom', 'miniproject-pytorch-AGES.ipynb', 'graphs-pytorch-AGES.ipynb', 'age_all_seqs.fasta', '.ipynb_checkpoints', 'AGP.data.biom.filtered.ages.tsv', 'AGP_ages_seqs.fa']


In [4]:
df_meta = pd.read_csv('../AGP_ages.metadata.txt', sep='\t')

/tmp/ipykernel_172249/1104074757.py:1: DtypeWarning: Columns (4,5,7,8,13,14,15,16,19,20,21,22,24,25,26,28,29,30,31,32,36,39,42,45,46,48,49,51,53,55,57,58,59,61,62,63,65,66,67,68,69,70,71,72,73,74,75,76,78,79,80,86,87,88,89,90,92,95,96,97,99,100,101,102,103,105,106,107,109,110,111,112,117,118,120,121,122,124,125,127,129,133,135,136,139,143,144,145,148,150,151,152,153,154,155,156,157,158,159,160,162,163,164,165,167,168,169,170,171,173,175,181,183,186,187,188,189,191,192,193,194,196,198,199,201,202,203,204,205,206,207,208,210,211,212,214,215,217,218,223,226,227,228,231,232,234,237,238,239,240,242,243,245,247,250,251,252,253,255,258,259,261,262,264,265,268,269,270,271,272,276,277,278,279,280,281,283,284,287,289,290,292,293,294,295,297,298,300,301,302,303,306,307,308,309,311,313,314,315,316,317,319,320,324,326,329,330,335,337,338,339,340,343,344,345,346,349,350,351,352,354,356,359,360,364,365,367,372,373,376,377,378,379,381,385,386,387,388,389,390,391,392,395,396,397,400,401,403,404,406,407

In [5]:
df_meta = df_meta.set_index("#SampleID")

In [6]:
df_meta.loc["10317.000076497.130064","age_cat"]

'50s'

In [10]:
#loading the reference phylogeny which we will use to identify the RELATIONSHIP between the abundance of the different microbes (sequences)

In [8]:
gg_phylogeny = TreeNode.read("../../2024.09.phylogeny.asv.nwk") #for now you don't actually need to load it

In [11]:
mats = pickle.load(open("../MATRICES_AGES.pickle","rb")) #these are pre-computed phylogenetic distances between sequences

In [12]:
mats.keys()

dict_keys(['TreeDistMatrix'])

In [14]:
dist_mat = mats["TreeDistMatrix"]

## Now, we are loading our abundance table, which contains the relative abundance of each microbe in each sample.

In [13]:
df_AGES = pd.read_csv('AGP.data.biom.filtered.ages.tsv',sep='\t')

In [15]:
df_AGES

,#OTU ID,10317.000076497.130064,10317.000020461.131460,10317.000035170.131134,10317.000101287.130337,10317.000112756.134923,10317.000048329.131418,10317.X00215040.147023,10317.000051537.131793,10317.000003425.129019,...,10317.000049792.131774,10317.000107692.132367,10317.000108522.134138,10317.000026461.128189,10317.000084679.127418,10317.000059044.131202,10317.000014949.129728,10317.000032829.129190,10317.000022172.131460,10317.000058378.130092
0,TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTG...,0.0,6.0,35.0,0.0,0.0,14.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,373.0,4.0,13.0,0.0,0.0,0.0
2,AACGTAGGTCACAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGAG...,0.0,0.0,0.0,65.0,0.0,0.0,139.0,0.0,0.0,...,0.0,25.0,2.0,0.0,473.0,123.0,0.0,0.0,10.0,0.0
3,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGAG...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CACGGGGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGCG...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2785,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,1.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,3.0,1.0,0.0,2.0,0.0
2786,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2787,TACGTAGGGCGCGAGCGTTGTCCGGATTTATTGGGCGTAAAGAGCT...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2788,TACGTAGGGGGCGAGCGTTATCCGGATTCATTGGGCGTAAAGCGCG...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df_meta

,non_food_allergies_drug_eg_penicillin,tonsils_removed,pm_shared_who_posteddiscussed_on_data_sharing_platform_ie_open_humans,vioscreen_m_meat,longitude,covid_worried_sleep,covid_happened_to_you_reduced_ability_to_earn_money,cancer_treatments_targeted_medication_therapy,pm_participation_reason_interest_in_contributing_to_science,fermented_produce_commercial_fermented_breadsourdoughinjera,...,vioscreen_pfa205,fermented_produce_commercial_cottage_cheese,alcohol_types_v2_sake,vioscreen_sfa40,vioscreen_water,specialized_diet_fodmap,covid_sleep_pattern,depression_bipolar_schizophrenia,covid_chronic_conditions_frequent_or_very_bad_headaches,mental_illness_type_ptsd_post_traumatic_stress_disorder
#SampleID,,,,,,,,,,,,,,,,,,,,,
10317.000076497.130064,False,No,NaN,NaN,1,not provided,False,NaN,NaN,false,...,NaN,false,NaN,NaN,NaN,False,not provided,not provided,False,False
10317.000020461.131460,True,No,NaN,NaN,-1,not provided,False,NaN,NaN,false,...,NaN,false,NaN,NaN,NaN,False,not provided,I do not have this condition,False,False
10317.000035170.131134,False,Yes,NaN,NaN,-88,not provided,False,NaN,NaN,false,...,NaN,false,NaN,NaN,NaN,False,not provided,"Diagnosed by a medical professional (doctor, p...",False,False
10317.000101287.130337,False,No,NaN,NaN,-114,not provided,False,NaN,NaN,false,...,NaN,false,NaN,NaN,NaN,False,not provided,not provided,False,False
10317.000112756.134923,False,not provided,NaN,NaN,-123,not provided,False,NaN,NaN,false,...,NaN,false,NaN,NaN,NaN,False,not provided,not provided,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000059044.131202,False,Yes,NaN,NaN,1,not provided,false,NaN,NaN,false,...,NaN,false,NaN,NaN,NaN,False,not provided,not provided,false,false
10317.000014949.129728,False,No,NaN,NaN,-122,not provided,false,NaN,NaN,false,...,NaN,false,NaN,NaN,NaN,False,not provided,not provided,false,false
10317.000032829.129190,False,No,NaN,NaN,-1,not provided,false,NaN,NaN,false,...,NaN,false,NaN,NaN,NaN,False,not provided,I do not have this condition,false,false


#### our df_ages contains our raw X (the vectors on which we will perform the prediction). Now we need to extract our Y (the labels associated with each vector) from our metadata file

In [17]:
#the classes for the variable age_cat are the following: ['20s','30s','40s','50s','60s'] 

In [18]:
labels = df_meta["age_cat"]

In [19]:
labels

#SampleID
10317.000076497.130064    50s
10317.000020461.131460    50s
10317.000035170.131134    40s
10317.000101287.130337    30s
10317.000112756.134923    60s
                         ... 
10317.000059044.131202    60s
10317.000014949.129728    40s
10317.000032829.129190    40s
10317.000022172.131460    40s
10317.000058378.130092    40s
Name: age_cat, Length: 24938, dtype: object

## Next steps

Now, we have our labels (the class labels corresponding to each sample ID), and the abundance of each microbe. So we are at the point where we start feature engineering.

1. we will need to normalize the abundance: you can divide each value of the abundance table by the total sum of microbes observed in that sample.
2. The order within each abundance vector will not matter for some methods, but will matter for others. For methods for which it matters, you will want to reorder the rows of the table via hierarchical clustering, using the distances in the distance matrix. The order should be fixed for all vectors.
3. We will need to convert the two dataframes into machine-learning compatible data types, so make sure the labels are in the same order as the columns of the abundance table, then convert to a dataset of X and Y, where X is a list of vectors, and Y is a vector where each index matches an index of X. You will probably need to convert the class labels to integers.
4. Then I guess we could start by trying to see if there is a signal with a very basic method, random forests, using the scikit-learn library ?
5. Then, we could try XGBoost (using the XGBoost Library).

For each, try many parameters to see if you get any significant signal (significantly more than 20%

Then we meet to see what results we got